# Python Programming for Spatial Data Analysis

Author: Zhan Zhao, PhD (zhanzhao@hku.hk)

Date: 09/06/2023

Prepared as part of the "3D Modeling and Visualization" executive course for Lands Department, HKSAR Government.

In this tutorial, we will learn some basic functinalities of three specialized Python packages for spatial data analysis:


*   **GeoPandas** for spatial data analysis
*   **OSMnx** for street network analysis
*   **Kepler.gl** for spatial data visualization


## 0. Setup

Install relevant Python packages: geopandas, osmnx, and keplergl.

In [ ]:
!pip install geopandas
!pip install osmnx
!pip install keplergl

Import relevant Python packages:

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import geopandas as gpd
import osmnx as ox
import keplergl

In Google Colab, to read files from Google Drive, we need to first mount Google Drive so that Google Colab can access files in Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 1. Building Data Analysis using GeoPandas

In this task, we will process building data in the Central and Western (CW) district in Hong Kong, using the **geopandas** package.

### 1.1 Obtain CW district boundary

First, we need to read the HK district boundary file, which can be downloaded in [Hong Kong GeoData Store](https://geodata.gov.hk/gs/datasets). We can use geopandas to read the geojson file into a GeoDataFrame.

In [ ]:
districts = gpd.read_file("/content/drive/MyDrive/Data/HK_Buildings/DCD.json")
districts.head()

In [ ]:
districts.plot()

Then, we can filter the GeoDataFrame to get only the CW district.

In [ ]:
cw = districts[districts['AREA_CODE'] == 'CW'].reset_index(drop=True)
cw.head()

In [ ]:
ax = districts.plot()
cw.plot(ax=ax, color='red')

### 1.2 Obtain Hong Kong building data

Again, we can download the building structure data from [Hong Kong GeoData Store](https://geodata.gov.hk/gs/datasets), and import the file in a similar way as Step 1.1. However, since there are many buildings in Hong Kong, the file size is big and this step can take a few minutes.

In [ ]:
buildings = gpd.read_file("/content/drive/MyDrive/Data/HK_Buildings/BUILDING_STRUCTURE.json")
buildings.head()

In [ ]:
buildings.shape

### 1.3 Filter the buildings in CW

To achieve this, we can use spatially join the HK buildings GeoDataFrame and the CW district GeoDataFrame. By default, the `sjoin` function in geopandas uses "inner join" based on the "intersects" option.

In [ ]:
buildings_cw = gpd.sjoin(buildings, cw)
buildings_cw = buildings_cw.drop(
    columns=['index_right','OBJECTID_right', 'CSDI_ADMIN_AREA_ID', 'AREA_TYPE',
             'AREA_ID', 'AREA_CODE', 'NAME_TC', 'NAME_EN', 'DATA_OWNER',
             'BEGIN_LIFESPAN', 'END_LIFESPAN', 'SHAPE_Length_right', 'SHAPE_Area_right']
    ).rename(columns={"SHAPE_Length_left": "SHAPE_Length", "SHAPE_Area_left": "SHAPE_Area"})
buildings_cw.shape

In [ ]:
buildings_cw.head()

If the data needs to be reused, it is good practice to save the data. In this case, we can save the filtered building data in CW into a GeoJSON file.

In [ ]:
buildings_cw.to_file("/content/drive/MyDrive/Data/HK_Buildings/buildings_cw.json", driver='GeoJSON')

### 1.4 Plot building data in CW

We can read the saved "buildings_cw.json" file if it exists.

In [ ]:
#buildings_cw = gpd.read_file("/content/drive/MyDrive/Data/HK_Buildings/buildings_cw.json")

In [ ]:
buildings_cw.plot(figsize=(16, 8))

Now, let us plot the buildings based on their "TOPHEIGHT". Note that some buildings do not have a valid value for "TOPHEIGHT". Therefore, we need to first filter out these observations.

In [ ]:
buildings_cw_with_height = buildings_cw[buildings_cw['TOPHEIGHT'].notnull()]
buildings_cw_with_height.shape

With the filtered data, we can color the building polygons based on their height.

In [ ]:
buildings_cw_with_height.plot(column='TOPHEIGHT', legend=True, figsize=(16, 8))

## 2. Street Network Analysis using OSMnx

In this task, we will perform some basic analysis on street network data for the CW district of Hong Kong. We will use **OSMnx** to obtain the street network data from [OpenStreetMap](https://www.openstreetmap.org/).

### 2.1 Coordinated Reference Systems (CRS) conversion

The geospatial data provided by the HK government in usually in **EPSG:2326**, which is a projected coordinate system suitable within the spatial boundary of HK. However, many internationally standard datasets (such as OpenStreetMap) uses **EPSG:4326** (or WGS 84). Therefore, to query street network data from OpenStreetMap, we need to first convert the CRS of the CW district boundary to WGS 84.

In [ ]:
cw.crs

In [ ]:
cw['geometry'][0]

In [ ]:
cw_WGS84 = cw.to_crs("epsg:4326")
cw_WGS84.crs

In [ ]:
cw_WGS84['geometry'][0]

### 2.2 Download street network data from OpenStreetMap

Once we define the CW district boundary in WGS84, we can use it to query the street network data from OpenStreetMap through OSMnx.

In [ ]:
cw_streets = ox.graph_from_polygon(cw_WGS84['geometry'][0])

In [ ]:
ox.plot_graph(cw_streets)

### 2.3 Basic analysis of the street network in CW

Let us first show some summary statistics about the street network in CW.

In [ ]:
stats = ox.basic_stats(cw_streets)
pd.Series(stats)

Then, we can visualize the street network and color the nodes (i.e. intersections) based on the number of connected street segments.

In [ ]:
nc_street_count = ox.plot.get_node_colors_by_attr(cw_streets, "street_count", cmap="YlGnBu")

fig, ax = ox.plot_graph(
    cw_streets,
    node_color=nc_street_count,
    node_size=5,
    node_zorder=2,
    edge_linewidth=0.2,
    edge_color="w"
)

## 3. Spatial Data Visualization using Kepler.gl

In this task, we will plot the buildings in CW using Kepler.gl, and explore its various functions.

Note that, in Google Colab, support for third party widgets (such as Kepler.gl) needs to be enabled separately.

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

### 3.1 Basic plot

In [ ]:
cw_map = keplergl.KeplerGl(height=600)
cw_map.add_data(data=buildings_cw_with_height.copy(), name="Buildings in CW")
cw_map

### 3.2 Explore more functionalities in Kepler.gl

Dr Jiali Zhou will introduce more functionalities in Kepler.gl without programming.